In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
# !pip install folium
import folium
# !pip install geopy 
from geopy.geocoders import Nominatim

# Fetching and setting up of data


In [38]:
india_df = pd.DataFrame()
data = requests.get('https://www.latlong.net/category/cities-102-15.html').text
soup = BeautifulSoup(data, 'html.parser')
table = soup.find_all('table')[0] 
dfdummy=pd.read_html(str(table))[0]
dfdummy

,Place Name,Latitude,Longitude
0,"Nanjangud, Mysore, Karnataka, India",12.120000,76.680000
1,"Chittorgarh, Rajasthan, India",24.879999,74.629997
2,"Ratnagiri, Maharashtra, India",16.994444,73.300003
3,"Goregaon, Mumbai, Maharashtra, India",19.155001,72.849998
4,"Pindwara, Rajasthan, India",24.794500,73.055000
5,"Raipur, Chhattisgarh, India",21.250000,81.629997
6,"Gokak, Karnataka, India",16.166700,74.833298
7,"Lucknow, Uttar Pradesh, India",26.850000,80.949997
8,"Delhi, the National Capital Territory of Delhi...",28.610001,77.230003
9,"Mumbai, Maharashtra, India",19.076090,72.877426


In [51]:
data = {'City':  dfdummy["Place Name"].tolist(),
        'Latitude': dfdummy["Latitude"].tolist(),
         'Longitude':dfdummy["Longitude"].tolist()
        }
india_df=pd.DataFrame(data, columns = ['City','Latitude','Longitude'])
india_df.head()

,City,Latitude,Longitude
0,"Nanjangud, Mysore, Karnataka, India",12.120000,76.680000
1,"Chittorgarh, Rajasthan, India",24.879999,74.629997
2,"Ratnagiri, Maharashtra, India",16.994444,73.300003
3,"Goregaon, Mumbai, Maharashtra, India",19.155001,72.849998
4,"Pindwara, Rajasthan, India",24.794500,73.055000


In [52]:
india_df['State'] = india_df['City'].apply(lambda x: x.split(',')[1] )
india_df['City'] = india_df['City'].apply(lambda x: x.split(',')[0] )
india_df.head()

,City,Latitude,Longitude,State
0,Nanjangud,12.120000,76.680000,Mysore
1,Chittorgarh,24.879999,74.629997,Rajasthan
2,Ratnagiri,16.994444,73.300003,Maharashtra
3,Goregaon,19.155001,72.849998,Mumbai
4,Pindwara,24.794500,73.055000,Rajasthan



# Use FourSquare API to explore venues

In [56]:
CLIENT_ID = 'DMGPTKDJFZ5YIWWLV51O5TPGH3UJQJX0B5XCGGAMAEK1EF4O' # your Foursquare ID
CLIENT_SECRET = 'K3JFSEMK40EG0NRQFSEYY41S5IL1VBO5NLT4NSPPEUJAP5BL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DMGPTKDJFZ5YIWWLV51O5TPGH3UJQJX0B5XCGGAMAEK1EF4O
CLIENT_SECRET:K3JFSEMK40EG0NRQFSEYY41S5IL1VBO5NLT4NSPPEUJAP5BL


In [57]:
radius = 500
LIMIT = 100

venues = []

for lat, long, city, state in zip(india_df['Latitude'], india_df['Longitude'], india_df['City'], india_df['State']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            city, 
            state,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

        
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['City', 'State', 'CityLatitude', 'CityLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head() 

(178, 8)


,City,State,CityLatitude,CityLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Nanjangud,Mysore,12.120000,76.680000,Apollo Pharmacy,12.120657,76.678324,Pharmacy
1,Chittorgarh,Rajasthan,24.879999,74.629997,Cantabil,24.882313,74.628812,Clothing Store
2,Goregaon,Mumbai,19.155001,72.849998,Vasu Sandwitch,19.157017,72.846070,Sandwich Place
3,Goregaon,Mumbai,19.155001,72.849998,Subway,19.155942,72.853372,Sandwich Place
4,Goregaon,Mumbai,19.155001,72.849998,Kiran Fast Food Corner,19.155983,72.851569,Fast Food Restaurant


In [67]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 76 uniques categories.


In [98]:
# check if the results contain "Clothing Store"
"Hotel" in venues_df['VenueCategory'].unique()
"Motel" in venues_df['VenueCategory'].unique()

True

In [59]:
# one hot encoding
india_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
india_onehot['City'] = venues_df['City'] 

# move city column to the first column
fixed_columns = [india_onehot.columns[-1]] + list(india_onehot.columns[:-1])
india_onehot = india_onehot[fixed_columns]
india_onehot.head()

,City,ATM,Accessories Store,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Bakery,Bank,Bar,...,Shopping Mall,Smoke Shop,Snack Place,Supermarket,Tailor Shop,Thrift / Vintage Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Zoo
0,Nanjangud,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Chittorgarh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Goregaon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Goregaon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Goregaon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
india_grouped = india_onehot.groupby('City').mean().reset_index()
india_grouped.head()

,City,ATM,Accessories Store,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Bakery,Bank,Bar,...,Shopping Mall,Smoke Shop,Snack Place,Supermarket,Tailor Shop,Thrift / Vintage Store,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Zoo
0,Alipurduar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Ambernath,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Barh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Basirhat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,Batumi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.130435,0.0,0.0


In [100]:
len(india_grouped[india_grouped["Hotel"] > 0])

8

In [83]:
hotel_df = india_grouped[["City","Hotel"]]
hotel_df.head()

,City,Hotel
0,Alipurduar,0.000000
1,Ambernath,1.000000
2,Barh,0.000000
3,Basirhat,0.000000
4,Batumi,0.130435


# Clustering

In [92]:
# set number of clusters
toclusters = 3

to_clustering = hotel_df.drop(["City"], 1)
# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [93]:
final_df = hotel_df.copy()

# add clustering labels
final_df["Cluster Labels"] = kmeans.labels_
final_df.head()

,City,Hotel,Cluster Labels
0,Alipurduar,0.000000,0
1,Ambernath,1.000000,1
2,Barh,0.000000,0
3,Basirhat,0.000000,0
4,Batumi,0.130435,0


In [94]:
final_df = final_df.join(venues_df.set_index("City"), on="City")

print(final_df.shape)
final_df.head()

(178, 10)


,City,Hotel,Cluster Labels,State,CityLatitude,CityLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alipurduar,0.000000,0,West Bengal,26.491890,89.527100,Doars mountain,26.488700,89.527700,Diner
1,Ambernath,1.000000,1,Maharashtra,19.186354,73.191948,Hotel Krishna Palace,19.189335,73.194900,Hotel
2,Barh,0.000000,0,Bihar,25.477585,85.709091,Crystal Palace,25.474192,85.708664,Bar
3,Basirhat,0.000000,0,West Bengal,22.657402,88.867180,Apollo Pharmacy,22.660578,88.867097,Pharmacy
4,Batumi,0.130435,0,Adjara,41.643414,41.639900,Hotel Marani,41.642824,41.641061,Hotel


In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# Examine Clusters

In [96]:
final_df.loc[final_df['Cluster Labels'] == 1]

,City,Hotel,Cluster Labels,State,CityLatitude,CityLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
1,Ambernath,1.0,1,Maharashtra,19.186354,73.191948,Hotel Krishna Palace,19.189335,73.194900,Hotel
50,Suri,1.0,1,West Bengal,23.905445,87.524620,Hotel SAGAR,23.904972,87.526749,Hotel


In [97]:
final_df.loc[final_df['Cluster Labels'] == 0]

,City,Hotel,Cluster Labels,State,CityLatitude,CityLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alipurduar,0.000000,0,West Bengal,26.491890,89.527100,Doars mountain,26.488700,89.527700,Diner
2,Barh,0.000000,0,Bihar,25.477585,85.709091,Crystal Palace,25.474192,85.708664,Bar
3,Basirhat,0.000000,0,West Bengal,22.657402,88.867180,Apollo Pharmacy,22.660578,88.867097,Pharmacy
4,Batumi,0.130435,0,Adjara,41.643414,41.639900,Hotel Marani,41.642824,41.641061,Hotel
4,Batumi,0.130435,0,Adjara,41.643414,41.639900,Дом Хинкали Калакури,41.642743,41.636833,Restaurant
4,Batumi,0.130435,0,Adjara,41.643414,41.639900,Plaza Fitness Club,41.646326,41.643409,Gym
4,Batumi,0.130435,0,Adjara,41.643414,41.639900,Sazandari,41.643463,41.640622,Restaurant
4,Batumi,0.130435,0,Adjara,41.643414,41.639900,Salkhino,41.643378,41.638640,Caucasian Restaurant
4,Batumi,0.130435,0,Adjara,41.643414,41.639900,Ci Gusta Batumi,41.642064,41.637459,Italian Restaurant
4,Batumi,0.130435,0,Adjara,41.643414,41.639900,Porto Franco | პორტო ფრანკო,41.647892,41.640296,Café


In [101]:
final_df.loc[final_df['Cluster Labels'] == 2]

,City,Hotel,Cluster Labels,State,CityLatitude,CityLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
15,Gondal,0.5,2,Gujarat,21.961946,70.792297,Old Market Yard,21.961990,70.792736,Market
15,Gondal,0.5,2,Gujarat,21.961946,70.792297,Orchard Heritage Palace,21.964382,70.794531,Hotel
15,Gondal,0.5,2,Gujarat,21.961946,70.792297,Orchard Palace,21.964746,70.795048,Hotel
15,Gondal,0.5,2,Gujarat,21.961946,70.792297,dreamland Restaurant,21.961724,70.797138,Indian Restaurant
39,Port Blair,0.5,2,Andaman and Nicobar Islands,11.623377,92.726486,Divyam Talkies,11.622831,92.725290,Movie Theater
39,Port Blair,0.5,2,Andaman and Nicobar Islands,11.623377,92.726486,Hotel De La’mour,11.623904,92.728567,Hotel
53,Tirumangalam,0.5,2,Tamil Nadu,9.823619,77.986565,SivaMatrimony,9.825441,77.984405,Astrologer
53,Tirumangalam,0.5,2,Tamil Nadu,9.823619,77.986565,VSR Lodge,9.825361,77.983821,Hotel


### Observations
Most of Hotels are in Cluster 2 which is around Gujarat and Tamil Nadu and lowest (close to zero) in Cluster 1 areas which are West Bengal and Maharashtra. Looking at nearby venues, it seems Cluster 1 might be a good location as there are not a lot of Hotels in these areas. Therefore, this project recommends the entrepreneur to open a hotel in these locations with little to no competition. Nonetheless, if the food is authentic, affordable and good taste, I am confident that it will have great following everywhere =)